<a href="https://colab.research.google.com/github/IsaacX777/MachineLearning/blob/main/bean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
import numpy as np

In [39]:
# Upload data files to Google Colab
from zipfile import ZipFile
file_name = 'dataset.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()

In [40]:
# Read data from CSV files
df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')

In [41]:
# Prepare images
def load_images(df):
  images = []
  labels = df['category']
  for image_path in df['image:FILE']:
    img = Image.open(image_path)
    img = img.resize((128, 128))
    img = np.array(img) / 255.0
    images.append(img)
  encoder = LabelEncoder()
  labels = encoder.fit_transform(labels)
  return np.array(images), np.array(labels)

In [42]:
# Data augmentation for reduced overfitting
data_augmentation = keras.Sequential([
    keras.layers.Input(shape=(128, 128, 3)),
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1),
])

In [43]:
# Setup images for training, testing, and validation
images, labels = load_images(df)
val_images, val_labels = load_images(val_df)
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

In [48]:
# Set up model
model = keras.Sequential([
    data_augmentation,
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

In [49]:
# Train model
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, validation_data=(val_images, val_labels))

Epoch 1/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.3767 - loss: 1.1995 - val_accuracy: 0.5789 - val_loss: 0.9725
Epoch 2/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.6099 - loss: 0.8866 - val_accuracy: 0.5865 - val_loss: 0.8566
Epoch 3/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.6036 - loss: 0.8196 - val_accuracy: 0.7068 - val_loss: 0.6273
Epoch 4/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.6760 - loss: 0.6883 - val_accuracy: 0.7293 - val_loss: 0.5642
Epoch 5/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.7318 - loss: 0.6099 - val_accuracy: 0.6992 - val_loss: 0.6172


In [50]:
# Model evaluation
test_loss = model.evaluate(test_images, test_labels)
test_loss

7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 405ms/step - accuracy: 0.7360 - loss: 0.6227


[0.6066210269927979, 0.7439613342285156]